In [ ]:
spark.stop()

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType, DateType

# Список драйверов (как у вас)
drivers = [
    "/opt/spark/external-jars/hadoop-aws-3.3.4.jar",
    "/opt/spark/external-jars/aws-java-sdk-bundle-1.12.262.jar",
    "/opt/spark/external-jars/wildfly-openssl-1.0.7.Final.jar",
    "/opt/spark/external-jars/postgresql-42.6.0.jar",
    "/opt/spark/external-jars/spark-sql-kafka-0-10_2.12-3.5.0.jar",
    "/opt/spark/external-jars/kafka-clients-3.2.0.jar",
    "/opt/spark/external-jars/spark-token-provider-kafka-0-10_2.12-3.5.0.jar",
    "/opt/spark/external-jars/commons-pool2-2.11.1.jar"
]

# Создаем Spark сессию (без лишних конфигов)
spark = (SparkSession.builder
         .appName("KafkaStreamingToPostgres")
         .master("spark://spark-master:7077")
         .config("spark.sql.shuffle.partitions", 5)
         .config("spark.jars", ",".join(drivers))
         .getOrCreate())

spark.sparkContext.setLogLevel("WARN")

print("✅ Spark сессия создана")

# 1. Читаем справочники из MinIO (кешируем их для производительности)
print("\n📁 Загружаем справочники...")
events_df = (spark.read
             .format("parquet")
             .load("s3a://library/kafka-enrich/event_list.parquet"))
events_df.cache().count()
print(f"Справочник событий: {events_df.count()} записей")
events_df.show()

groups_df = (spark.read
             .format("parquet")
             .load("s3a://library/kafka-enrich/group_list.parquet"))
groups_df.cache().count()
print(f"\nСправочник групп: {groups_df.count()} записей")
groups_df.show()

# 2. Определяем схему для парсинга JSON
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("date", StringType(), True),
    StructField("event_date", StringType(), True),
    StructField("event_id", IntegerType(), True),
    StructField("username", StringType(), True),
    StructField("group_id", IntegerType(), True),
    StructField("value", DoubleType(), True)
])

# 3. СОЗДАЕМ СТРИМИНГ вместо батч-чтения
print("\n📨 Подключаемся к потоку Kafka...")
kafka_stream = (spark.readStream
                .format("kafka")
                .option("kafka.bootstrap.servers", "kafka:9092")
                .option("subscribe", "user-events")
                .option("startingOffsets", "latest")  # Только новые сообщения
                .option("failOnDataLoss", "false")
                .option("maxOffsetsPerTrigger", 10)   # Контроль скорости
                .load())

# 4. Парсим JSON как в вашем коде
parsed_stream = (kafka_stream
                 .select(from_json(col("value").cast("string"), schema).alias("data"))
                 .select("data.*")
                 .withColumn("date", col("date").cast(DateType()))
                 .withColumn("event_date", col("event_date").cast(TimestampType())))

print("✅ Стрим создан, схема:")
parsed_stream.printSchema()

# 5. Обогащаем данные (точно как в вашем коде)
enriched_stream = (parsed_stream
                   .join(events_df, on="event_id", how="left")
                   .join(groups_df, on="group_id", how="left"))

# 6. Готовим финальный датасет (как у вас)
final_stream = enriched_stream.select(
    "id", "date", "event_date", 
    "event_id", 
    "event_name",  
    "username", 
    "group_id",
    "group_name",
    "value"
)

print("\n📊 Финальная схема потока:")
final_stream.printSchema()

# 7. Функция для записи каждого микробатча в PostgreSQL
def write_to_postgres(batch_df, batch_id):
    """Записывает каждый микробатч в PostgreSQL"""
    try:
        # Используем точно такой же код записи, как в вашем рабочем примере
        batch_df.write \
            .mode("append") \
            .jdbc(url="jdbc:postgresql://postgres-db:5432/learn_base",
                  table="kafka_farm.user_events",
                  properties={
                      "user": "airflow",
                      "password": "airflow",
                      "driver": "org.postgresql.Driver"
                  })
        print(f"✅ Batch {batch_id}: записано {batch_df.count()} записей")
    except Exception as e:
        print(f"❌ Batch {batch_id}: ошибка - {e}")

# 8. ЗАПУСКАЕМ СТРИМИНГ
print("\n🚀 ЗАПУСК СТРИМИНГА В POSTGRESQL")
print("=" * 50)

query = (final_stream.writeStream
         .foreachBatch(write_to_postgres)
         .outputMode("append")
         .trigger(processingTime="10 seconds")  # Микробатчи каждые 10 секунд
         .option("checkpointLocation", "/tmp/spark-checkpoints/kafka-to-postgres")
         .start())

print("✅ Стриминг запущен и пишет в PostgreSQL!")
print("📝 Данные поступают в таблицу: kafka_farm.user_events")
print("\n🔍 КОМАНДЫ УПРАВЛЕНИЯ:")
print("-" * 50)
print("Проверить статус: query.isActive")
print("Посмотреть прогресс: query.recentProgress")
print("ОСТАНОВИТЬ: query.stop()")
print("=" * 50)

# Сразу покажем статус
print(f"\nСтатус: {'АКТИВЕН' if query.isActive else 'ОСТАНОВЛЕН'}")
print(f"ID стрима: {query.id}")

26/02/27 20:44:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


✅ Spark сессия создана

📁 Загружаем справочники...


26/02/27 20:44:29 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

Справочник событий: 5 записей
+--------+--------------------+
|event_id|          event_name|
+--------+--------------------+
|       5|Пожертвование сектам|
|       4|    Донатная помойка|
|       2|    Вложение в акции|
|       1|      Покупка крипты|
|       3|       Оплата товара|
+--------+--------------------+


Справочник групп: 5 записей
+--------+-----------+
|group_id| group_name|
+--------+-----------+
|       2|Иностранные|
|       3| Рептилойды|
|       1|      Новые|
|       5|   Platinum|
|       4|       Gold|
+--------+-----------+


📨 Подключаемся к потоку Kafka...
✅ Стрим создан, схема:
root
 |-- id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- event_date: timestamp (nullable = true)
 |-- event_id: integer (nullable = true)
 |-- username: string (nullable = true)
 |-- group_id: integer (nullable = true)
 |-- value: double (nullable = true)


📊 Финальная схема потока:
root
 |-- id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- ev

26/02/27 20:44:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
26/02/27 20:44:41 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
26/02/27 20:44:41 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
26/02/27 20:44:41 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
26/02/27 20:44:41 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
26/02/27 20:44:41 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


✅ Batch 0: записано 0 записей


✅ Batch 1: записано 9 записей
✅ Batch 2: записано 9 записей
✅ Batch 3: записано 9 записей
✅ Batch 4: записано 8 записей


✅ Batch 5: записано 9 записей
✅ Batch 6: записано 9 записей
✅ Batch 7: записано 9 записей
✅ Batch 8: записано 8 записей
✅ Batch 9: записано 9 записей
✅ Batch 10: записано 9 записей
✅ Batch 11: записано 9 записей
✅ Batch 12: записано 9 записей


In [2]:
query.stop()
print("Стриминг остановлен")

Стриминг остановлен


In [3]:
spark.stop()

# Мониторинг

In [ ]:
def quick_status(query):
    """Быстрая проверка статуса"""
    if query.recentProgress:
        last = query.recentProgress[-1]
        total = sum(p.get('numInputRows', 0) for p in query.recentProgress)
        print(f"✅ Стрим активен | Батч #{last.get('batchId', 0)}: {last.get('numInputRows', 0)} записей | "
              f"Всего: {total} записей за {len(query.recentProgress)} батчей | "
              f"Скорость: {last.get('processedRowsPerSecond', 0):.1f} rows/сек")
    else:
        print("✅ Стрим активен | Ожидание данных...")

# Использование
quick_status(query)

In [ ]:
def check_stream_status(query):
    """Проверяет статус стриминга и выводит доступную информацию"""
    print("=" * 60)
    print("📊 МОНИТОРИНГ СТРИМИНГА")
    print("=" * 60)
    
    # Основной статус
    print(f"\n✅ Активен: {query.isActive}")
    print(f"🆔 ID стрима: {query.id}")
    print(f"🆔 Run ID: {query.runId}")
    
    # Статус сообщение
    status = query.status
    print(f"\n📋 Статус: {status['message']}")
    
    # Информация о последнем прогрессе
    if query.recentProgress:
        last = query.recentProgress[-1]
        print(f"\n📈 ПОСЛЕДНИЙ БАТЧ (ID: {last.get('batchId', 'N/A')}):")
        print(f"   • Время: {last.get('timestamp', 'N/A')}")
        print(f"   • Записей обработано: {last.get('numInputRows', 0)}")
        print(f"   • Скорость входа: {last.get('inputRowsPerSecond', 0):.2f} rows/сек")
        print(f"   • Скорость обработки: {last.get('processedRowsPerSecond', 0):.2f} rows/сек")
        
        # Длительность этапов (из durationMs)
        if 'durationMs' in last:
            duration = last['durationMs']
            total_time = duration.get('triggerExecution', 0)
            print(f"\n   ⏱️  Длительность этапов (мс):")
            print(f"      • Общее время: {total_time} мс")
            print(f"      • Добавление батча: {duration.get('addBatch', 0)} мс")
            print(f"      • Получение оффсетов: {duration.get('latestOffset', 0)} мс")
            print(f"      • Планирование запроса: {duration.get('queryPlanning', 0)} мс")
            print(f"      • Коммит оффсетов: {duration.get('commitOffsets', 0)} мс")
        
        # Информация об источниках (партиции Kafka)
        if 'sources' in last and last['sources']:
            source = last['sources'][0]
            print(f"\n   📍 Источник Kafka:")
            print(f"      • Топик: {source.get('description', 'N/A')}")
            if 'metrics' in source:
                metrics = source['metrics']
                print(f"      • Отставание (max): {metrics.get('maxOffsetsBehindLatest', 0)}")
                print(f"      • Отставание (avg): {metrics.get('avgOffsetsBehindLatest', 0)}")
        
        # Суммарная статистика за все время
        total_rows = sum(p.get('numInputRows', 0) for p in query.recentProgress)
        total_batches = len(query.recentProgress)
        
        print(f"\n📊 СТАТИСТИКА ЗА ВСЕ ВРЕМЯ:")
        print(f"   • Всего батчей: {total_batches}")
        print(f"   • Всего записей: {total_rows}")
        
        if total_batches > 0:
            avg_rows = total_rows / total_batches
            print(f"   • Среднее записей/батч: {avg_rows:.1f}")
    else:
        print("\n⏳ Нет информации о прогрессе (возможно, стрим только что запущен)")
    
    # Проверка исключений
    if query.exception():
        print(f"\n❌ Исключение: {query.exception()}")
    
    print("=" * 60)

# Используем функцию
check_stream_status(query)

In [ ]:
# В Jupyter выполните:
print(f"Стрим активен: {query.isActive}")
print(f"ID стрима: {query.id}")

if query.recentProgress:
    print(f"Последний батч: {query.recentProgress[-1]['batchId']}")
    print(f"Всего батчей: {len(query.recentProgress)}")
else:
    print("Нет прогресса - стрим не получает данные")